# asyncio + ProcessPoolExecutor (spawn) using an importable worker module

Worker targets live in a real .py module on disk, so spawn can import them.


In [ ]:
from __future__ import annotations

import importlib
import sys
import uuid
from pathlib import Path
from tempfile import TemporaryDirectory

# Write a tiny importable module into a temporary directory.
# Use a unique module name to avoid collisions under xdist.
_temp_module_dir_ctx = TemporaryDirectory(
    prefix="nbast_worker_mod_",
    dir=str(Path.cwd()),
)
module_dir = Path(_temp_module_dir_ctx.name)
module_name = f"nbast_worker_mod_{uuid.uuid4().hex}"
module_path = module_dir / f"{module_name}.py"
module_path.write_text(
    "def cpu_fn(x):\n" "    return x * x\n",
    encoding="utf-8",
)

if str(module_dir) not in sys.path:
    sys.path.insert(0, str(module_dir))
importlib.invalidate_caches()
wm = importlib.import_module(module_name)

In [ ]:
import asyncio
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor


async def main() -> None:
    ctx = mp.get_context("spawn")
    loop = asyncio.get_running_loop()
    xs = list(range(10))
    with ProcessPoolExecutor(max_workers=2, mp_context=ctx) as ex:
        tasks = [loop.run_in_executor(ex, wm.cpu_fn, x) for x in xs]
        ys = await asyncio.gather(*tasks)
    assert ys == [x * x for x in xs]


try:
    await main()
finally:
    sys.modules.pop(module_name, None)
    if str(module_dir) in sys.path:
        sys.path.remove(str(module_dir))
    _temp_module_dir_ctx.cleanup()